# Istari AIAA Hands on Keyboard Event

## About this doc

This notebook demonstrates the Istari platform workflow for a Group 3 expendable UAV (tailless flying wing).
You will modify wing design parameters, run an nTop structural analysis, and verify compliance with system requirements.

**Goal**: Achieve the lowest Structure Weight while meeting all requirements.

In [ ]:
#@title User Data { display-mode: "form" }
!pip install istari-digital-client -q

from google.colab import userdata

#@markdown Your `ISTARI_PAT` should be stored in Colab Secrets (🔑 in sidebar)

ISTARI_ENVIRONMENT_URL = "https://fileservice-v2.stage.istari.app"
ISTARI_PAT = userdata.get('ISTARI_PAT')

#@markdown ---

NTOP_MODEL_ID = "5ddb114c-8f6d-4788-82d3-135d343131f1" #@param {type:"string"}
INPUT_JSON_MODEL_ID = "15686f36-9827-42b8-8ef0-91fa61c788d1" #@param {type:"string"}

print(f"✓ Ready" if ISTARI_PAT else "✗ PAT not found (check Colab Secrets)")

In [ ]:
#@title Wing Design Parameters { display-mode: "form" }

LOA_In = 99.9 #@param {type:"number"}
Span_In = 144.0 #@param {type:"number"}
Leading_Edge_Sweep_Inboard_deg = 46.5 #@param {type:"number"}
Leading_Edge_Sweep_Outboard_deg = 46.5 #@param {type:"number"}
Trailing_Edge_Sweep_Inboard_deg = -46.5 #@param {type:"number"}
Trailing_Edge_Sweep_Outboard_deg = 15.0 #@param {type:"number"}
Panel_Break_Span_Fraction = 0.3 #@param {type:"number"}

# Map to internal variable names
loa_in = LOA_In
span = Span_In
le_sweep_p1 = Leading_Edge_Sweep_Inboard_deg
le_sweep_p2 = Leading_Edge_Sweep_Outboard_deg
te_sweep_p1 = Trailing_Edge_Sweep_Inboard_deg
te_sweep_p2 = Trailing_Edge_Sweep_Outboard_deg
panel_break_span_pct = Panel_Break_Span_Fraction

In [ ]:
#@title Analyze and Verify { display-mode: "form" }
import json
import tempfile
import os
import time
import logging
from istari_digital_client.client import Client
from istari_digital_client.configuration import Configuration
from istari_digital_client import JobStatusName, NewSource
from istari_digital_client.exceptions import ConflictException

# === Connect to Istari ===
print("Connecting to Istari...")
client = Client(
    config=Configuration(
        registry_url=ISTARI_ENVIRONMENT_URL,
        registry_auth_token=ISTARI_PAT,
    )
)
ntop_model = client.get_model(NTOP_MODEL_ID)
input_json_model = client.get_model(INPUT_JSON_MODEL_ID)
print(f"✓ Connected ({ntop_model.display_name or ntop_model.name})")

# === Step 1: Update drone parameters ===
print(f"Updating drone parameters: LOA={loa_in}in, Span={span}in, LE Sweep={le_sweep_p1}°/{le_sweep_p2}°, TE Sweep={te_sweep_p1}°/{te_sweep_p2}°, Panel Break={panel_break_span_pct}")

input_json = {
    "inputs": [
        {"name": "LOA In", "type": "real", "units": "in", "value": loa_in},
        {"name": "Span", "type": "real", "units": "in", "value": span},
        {"name": "LE Sweep P1", "type": "real", "units": "deg", "value": le_sweep_p1},
        {"name": "LE Sweep P2", "type": "real", "units": "deg", "value": le_sweep_p2},
        {"name": "TE Sweep P1", "type": "real", "units": "deg", "value": te_sweep_p1},
        {"name": "TE Sweep P2", "type": "real", "units": "deg", "value": te_sweep_p2},
        {"name": "Panel Break Span %", "type": "real", "value": panel_break_span_pct},
        {"name": "MAIN PATH", "type": "file_path", "value": "./Outputs/"}
    ]
}

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    f.write(json.dumps(input_json, indent=2))
    temp_input_path = f.name

logging.getLogger('istari_digital_client').setLevel(logging.CRITICAL)

try:
    input_json_model = client.update_model(model_id=INPUT_JSON_MODEL_ID, path=temp_input_path)
except ConflictException:
    input_json_model = client.get_model(INPUT_JSON_MODEL_ID)
finally:
    logging.getLogger('istari_digital_client').setLevel(logging.WARNING)

os.unlink(temp_input_path)

# === Step 2: Run nTop analysis ===
print("Starting nTop analysis...")

source_ntop = NewSource(revision_id=ntop_model.revision.id, relationship_identifier="ntop_model")

run_job = client.add_job(
    model_id=input_json_model.id,
    function="@ntop:run_model",
    tool_name="ntopcl",
    tool_version="5.30",
    operating_system="RHEL 9",
    sources=[source_ntop],
)

last_status = None
while run_job.status.name not in [JobStatusName.COMPLETED, JobStatusName.FAILED]:
    time.sleep(5)
    run_job = client.get_job(run_job.id)
    if run_job.status.name != last_status:
        status_msg = str(run_job.status.name).replace("_", " ").lower()
        print(f"nTop analysis is {status_msg}")
        last_status = run_job.status.name

if run_job.status.name == JobStatusName.FAILED:
    print("❌ There was an error in the nTop analysis")
    raise Exception("nTop analysis failed")

print("✓ nTop analysis complete")

# === Step 3: Verify outputs ===
print("Verifying outputs against requirements...")

input_json_model = client.get_model(input_json_model.id)

output_values = {}
for artifact in input_json_model.artifacts:
    if artifact.name == "output.json":
        output_data = json.loads(artifact.read_text())
        if isinstance(output_data, list):
            for item in output_data:
                if isinstance(item, dict) and item.get("type") == "json":
                    output_values = item.get("value", {}).get("jsonObject", {})

fuel_volume = output_values.get("Fuel_Volume")
structure_weight = output_values.get("Structure_Weight")
surface_area = output_values.get("Surface_Area")

requirements = {
    "Structure_Weight": {"max": 300.0, "units": "lb"},
    "Fuel_Volume": {"min": 2500.0, "units": "mm³"}
}

weight_ok = structure_weight is not None and structure_weight <= requirements["Structure_Weight"]["max"]
fuel_ok = fuel_volume is not None and fuel_volume >= requirements["Fuel_Volume"]["min"]
all_pass = weight_ok and fuel_ok

print("\n" + "=" * 50)
print("            ANALYSIS COMPLETE")
print("=" * 50)
print(f"\n  Structure Weight: {structure_weight:.2f} lb (max: {requirements['Structure_Weight']['max']} lb) {'✅' if weight_ok else '❌'}" if structure_weight else "\n  Structure Weight: N/A")
print(f"  Fuel Volume: {fuel_volume:.2f} mm³ (min: {requirements['Fuel_Volume']['min']} mm³) {'✅' if fuel_ok else '❌'}" if fuel_volume else "  Fuel Volume: N/A")
print(f"  Surface Area: {surface_area:.2f} mm²" if surface_area else "  Surface Area: N/A")

print("\n" + "=" * 50)
if all_pass:
    print(f"✅ ALL REQUIREMENTS PASSED!")
    print(f"🎯 Your Structure Weight: {structure_weight:.2f} lb")
    print(f"   Can you get it lower while still passing?")
else:
    print("❌ REQUIREMENTS NOT MET")
    if not weight_ok:
        print("   - Structure Weight exceeds maximum!" if structure_weight else "   - Structure Weight not available")
    if not fuel_ok:
        print("   - Fuel Volume below minimum!" if fuel_volume else "   - Fuel Volume not available")
print("=" * 50)